In [1]:
#This jupyter notebok is for downloading data parallelly from s3 bucket to local file system:

In [2]:
import boto3
import botocore
import time
import os
import pandas as pd
from multiprocessing.pool import ThreadPool

In [3]:
data_link = "amazon-reviews-pds"
output_dir = "../Data2/"

s3 = boto3.client('s3')
response = s3.list_objects_v2(Bucket=data_link)

In [12]:
def downloadFile(fileItem):
    head, file = os.path.split(fileItem)
    filename = output_dir + file
    s = time.time()
    s3.download_file(Bucket=data_link, Key=fileItem, Filename=filename)
    e = time.time()
    print("Downloaded file {} in {} secs!".format(fileItem, e-s))
    return None

In [5]:
data = []
for fileItem in response['Contents']:
    if str(fileItem['Key']).startswith('tsv'):
        head, file = os.path.split(fileItem['Key'])
        if file != "":
            data.append(fileItem)

In [6]:
df = pd.DataFrame(data)
df = df.sort_values('Size').reset_index()

In [47]:
batches = {}
batch_num = 0
data = []
size_sum = 0
for i in range(df.shape[0]):
    size_sum += int(df['Size'][i])
    data.append(df['Key'][i])
    if (size_sum/1024/1024 > 200) or (len(data) > 8):
        if len(data) > 1:
            data.pop()
            batches[batch_num] = data
            size_sum = int(df['Size'][i])
            data = [df['Key'][i]]
        else:
            batches[batch_num] = data
            size_sum = 0
            data = []
        batch_num += 1
batches[batch_num] = data

In [13]:
start = time.time()
for batch in batches.values():
    results = ThreadPool(len(batch)).imap_unordered(downloadFile, batch)
    for result in results:
        while result != None:
            continue
end = time.time()
print("Total Time for Downloading files is {} secs!".format(end-start))

Downloaded file tsv/index.txt in 0.2660233974456787 secs!
Downloaded file tsv/sample_fr.tsv in 0.41094112396240234 secs!
Downloaded file tsv/sample_us.tsv in 0.4139399528503418 secs!
Downloaded file tsv/amazon_reviews_us_Gift_Card_v1_00.tsv.gz in 2.1560301780700684 secs!
Downloaded file tsv/amazon_reviews_us_Mobile_Electronics_v1_00.tsv.gz in 3.4980218410491943 secs!
Downloaded file tsv/amazon_reviews_us_Major_Appliances_v1_00.tsv.gz in 3.859261989593506 secs!
Downloaded file tsv/amazon_reviews_us_Digital_Software_v1_00.tsv.gz in 4.297243356704712 secs!
Downloaded file tsv/amazon_reviews_us_Personal_Care_Appliances_v1_00.tsv.gz in 5.450909852981567 secs!
Downloaded file tsv/amazon_reviews_us_Luggage_v1_00.tsv.gz in 16.355531215667725 secs!
Downloaded file tsv/amazon_reviews_us_Video_v1_00.tsv.gz in 17.057432651519775 secs!
Downloaded file tsv/amazon_reviews_multilingual_FR_v1_00.tsv.gz in 17.464447021484375 secs!
Downloaded file tsv/amazon_reviews_us_Software_v1_00.tsv.gz in 17.9917035